In [2]:
import pandas as pd

In [20]:
ds_path = r"C:\Yahia\python\ML\data\GermanyUsedCars.zip"
dfo = pd.read_csv(ds_path, index_col=0)

In [26]:
# dfo.year.value_counts()
# dfo.query(int(dfo.year)) != 
dfo.year.astype(int, errors='ignore')

0         1995
1         1995
2         1995
3         1995
4         1996
          ... 
251074    2023
251075    2023
251076    2023
251077    2023
251078    2023
Name: year, Length: 251079, dtype: object

In [21]:
dfo.columns

Index(['brand', 'model', 'color', 'registration_date', 'year', 'price_in_euro',
       'power_kw', 'power_ps', 'transmission_type', 'fuel_type',
       'fuel_consumption_l_100km', 'fuel_consumption_g_km', 'mileage_in_km',
       'offer_description'],
      dtype='object')

### Data Cleansing

In [76]:
df = dfo.assign(year_e = pd.to_numeric(dfo.year, errors='coerce'))
# suppress records with invalid year 
df = df.loc[~((df.year_e.isna()) | (df.year_e > 2023)) ]

In [78]:
df.year_e.value_counts()

year_e
2019.0    29225
2018.0    24095
2023.0    21097
2022.0    20653
2017.0    18940
2020.0    18566
2021.0    16022
2016.0    15072
2015.0    12712
2014.0    10623
2013.0     9325
2012.0     7992
2011.0     7384
2009.0     6431
2010.0     6043
2008.0     4847
2007.0     4018
2006.0     3973
2005.0     3399
2004.0     2649
2003.0     2101
2002.0     1581
2001.0     1216
2000.0      963
1999.0      744
1998.0      469
1997.0      343
1996.0      227
1995.0      170
Name: count, dtype: int64

In [ ]:
# dfo.head()
# dfo.brand.value_counts()
# dfo[['year', 'registration_date']].head()
df = dfo.assign(year_e = pd.to_numeric(dfo.year, errors='coerce'))
# df = dfo.assign(years_in_service = 2023-pd.to_numeric(dfo.year, errors='coerce'))
# dfo.year.astype(int, errors='ignore') - 2023
# df[['brand', 'year', 'mileage_in_km']].groupby(['brand', 'year']).sum()/2023-year
# 2023 - dfo.year.astype(int, errors='ignore')
# df.head()
# df.years_on_service
# df.years_in_service = 2023 - df.years_in_service
# df = df.loc[df.years_in_service >= 0]
df[['year', 'year_e']].value_counts()
# df[['year']].value_counts()


In [54]:
pd.to_numeric(dfo.year, errors='coerce').value_counts()


0         1995.0
1         1995.0
2         1995.0
3         1995.0
4         1996.0
           ...  
251074    2023.0
251075    2023.0
251076    2023.0
251077    2023.0
251078    2023.0
Name: year, Length: 251079, dtype: float64